In [64]:
import matplotlib.pyplot as plt
import numpy as np

In [65]:
weight_1_1, weight_2_1 = 0, 0
weight_1_2, weight_2_2 = 0, 0

bias_1, bias_2 = 0, 0


# Classifies the given fruit as either safe (0) or poisonous (1)
def classify(input_1, input_2):
    output_1 = input_1 * weight_1_1 + input_2 * weight_2_1 + bias_1
    output_2 = input_1 * weight_1_2 + input_2 * weight_2_2 + bias_2
    # Return 0 if output_1 is greater than output_2, otherwise return 1
    return 0 if output_1 > output_2 else 1

classify(1, 1)



1

In [66]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interactive

# Initial weights and biases
initial_params = {'weight_1_1': 1, 'weight_2_1': -1, 'weight_1_2': -1, 'weight_2_2': 1, 'bias_1': 0, 'bias_2': 0}

# Classify function with biases
def classify(input_1, input_2, weight_1_1, weight_2_1, weight_1_2, weight_2_2, bias_1, bias_2):
    output_1 = input_1 * weight_1_1 + input_2 * weight_2_1 + bias_1
    output_2 = input_1 * weight_1_2 + input_2 * weight_2_2 + bias_2
    return 0 if output_1 > output_2 else 1

# Update function for the interactive plot with biases
def update(weight_1_1, weight_2_1, weight_1_2, weight_2_2, bias_1, bias_2):
    grid_x, grid_y = np.meshgrid(np.linspace(-10, 10, 500), np.linspace(-10, 10, 500))
    classification = np.vectorize(classify)(grid_x, grid_y, weight_1_1, weight_2_1, weight_1_2, weight_2_2, bias_1, bias_2)
    
    plt.contourf(grid_x, grid_y, classification, levels=[-1, 0.5, 2], cmap='bwr', alpha=0.5)
    plt.scatter(x_coords, y_coords, c=['r' if cls == 1 else 'b' for cls in classes])
    plt.show()

# Generate random data points for illustration
np.random.seed(0)
x_coords = np.random.uniform(-10, 10, 100)
y_coords = np.random.uniform(-10, 10, 100)
classes = np.vectorize(classify)(x_coords, y_coords, **initial_params)

# Create interactive sliders including biases
interactive_plot = interactive(update,
                               weight_1_1=(-1.0, 1.0),
                               weight_2_1=(-1.0, 1.0),
                               weight_1_2=(-1.0, 1.0),
                               weight_2_2=(-2.0, 2.0),
                               bias_1=(-1.0, 1.0),
                               bias_2=(-1.0, 1.0))
output = interactive_plot.children[-1]
output.layout.height = '350px'  # Adjust the height to fit the plot

# Display the interactive plot with sliders
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='weight_1_1', max=1.0, min=-1.0), FloatSlider(value=0…

### layers

In [67]:
class Layer:
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size)
        self.biases = np.zeros(output_size)
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def tanh(self, x):
        e2x = np.exp(2*x)
        return (e2x - 1) / (e2x + 1)
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def activation(self, x):
        return self.sigmoid(x)
    
    def forward(self, inputs):
        result = []
        for weights_row, bias in zip(self.weights, self.biases):
            sum_result = 0
            for weight, inpt in zip(weights_row, inputs):
                sum_result += weight * inpt
            sum_result += bias
            result.append(self.activation(sum_result))
        return result

class NN:
    def __init__(self, layer_sizes):
        self.layers = []
        for i in range(len(layer_sizes) - 1):
            self.layers.append(Layer(layer_sizes[i], layer_sizes[i+1]))

    def forward(self, inputs):
        for layer in self.layers:
            inputs = layer.forward(inputs)
        return inputs

    def predict(self, inputs):
        outputs = self.forward(inputs)
        max_index = 0
        max_value = outputs[0]
        for i in range(1, len(outputs)):
            if outputs[i] > max_value:
                max_value = outputs[i]
                max_index = i
        return max_index


def binary_cross_entropy_loss(output_activation, expected_output):
    # Avoid division by zero
    output_activation = np.clip(output_activation, 1e-10, 1 - 1e-10)
    # Compute binary cross-entropy
    return -(
        expected_output * np.log(output_activation)
        + (1 - expected_output) * np.log(1 - output_activation)
    )


def cost(data_point, network):
    output_activation = network.forward(data_point["inputs"])
    # Assuming a single output neuron
    return binary_cross_entropy_loss(
        output_activation[0], data_point["expected_outputs"][0]
    )


def total_cost(data, network):
    total_cost = 0
    for data_point in data:
        total_cost += cost(data_point, network)
    return total_cost / len(data)

In [68]:
network = NN([2, 3, 2])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interactive, FloatSlider, VBox, HBox, Layout, Output

# Assuming your NN class and Layer class are defined as in the provided context


# Function to classify input using the neural network
def classify_network(input_1, input_2, network):
    return network.predict([input_1, input_2])


# Function to create an interactive plot for a given network
def interactive_plot(network, data):
    sliders = {}
    for layer_index, layer in enumerate(network.layers):
        # Create a slider for each weight
        for j in range(layer.weights.shape[0]):
            for k in range(layer.weights.shape[1]):
                slider_key = f"W{layer_index}_{j}_{k}"
                sliders[slider_key] = FloatSlider(
                    value=layer.weights[j, k],
                    min=-1.0,
                    max=1.0,
                    step=0.01,
                    description=f"W[{layer_index}][{j},{k}]",
                    continuous_update=False,
                    layout=Layout(width="400px"),
                )
        # Create a slider for each bias
        for j in range(layer.biases.shape[0]):
            slider_key = f"B{layer_index}_{j}"
            sliders[slider_key] = FloatSlider(
                value=layer.biases[j],
                min=-1.0,
                max=1.0,
                step=0.01,
                description=f"B[{layer_index}][{j}]",
                continuous_update=False,
                layout=Layout(width="400px"),
            )

    # Update function for the interactive plot
    def update(**kwargs):
        # Update network weights and biases based on the sliders
        for layer_index, layer in enumerate(network.layers):
            # Update weights
            for j in range(layer.weights.shape[0]):
                for k in range(layer.weights.shape[1]):
                    layer.weights[j, k] = kwargs[f"W{layer_index}_{j}_{k}"]
            # Update biases
            for j in range(layer.biases.shape[0]):
                layer.biases[j] = kwargs[f"B{layer_index}_{j}"]

        current_cost = total_cost(data, network)
        print(f"Current cost: {current_cost:.4f}")

        # Generate a grid of points and classify each point
        grid_x, grid_y = np.meshgrid(
            np.linspace(-10, 10, 500), np.linspace(-10, 10, 500)
        )
        classification = np.vectorize(lambda x, y: classify_network(x, y, network))(
            grid_x, grid_y
        )

        # Clear the current figure
        plt.clf()

        # Plot the decision boundary and data points
        plt.contourf(
            grid_x, grid_y, classification, levels=[-1, 0.5, 2], cmap="bwr", alpha=0.5
        )
        plt.scatter(x_coords, y_coords, c=["r" if cls == 1 else "b" for cls in classes])
        plt.xlim(-10, 10)
        plt.ylim(-10, 10)
        plt.show()

    # Create interactive plot with sliders
    interactive_plot = interactive(update, **sliders)
    slider_boxes = [
        VBox(
            [
                v
                for k, v in sliders.items()
                if k.startswith(f"W{layer_index}") or k.startswith(f"B{layer_index}")
            ]
        )
        for layer_index in range(len(network.layers))
    ]
    return HBox(slider_boxes + [interactive_plot.children[-1]])


# Generate non-linear data points for illustration
np.random.seed(0)
x_coords = np.random.uniform(-10, 10, 100)
y_coords = np.random.uniform(-10, 10, 100)
classes = np.where(x_coords**2 + y_coords**2 < 36, 0, 1)  # Circular decision boundary
# Assuming each data point should have two expected outputs to match the network's output layer
data = [
    {
        "inputs": np.array([x, y]),
        "expected_outputs": np.array([c, 1 - c]),
    }  # Example for two output nodes
    for x, y, c in zip(x_coords, y_coords, classes)
]
# Initialize the network with random weights and biases
network = NN([2, 3, 2])


# Create and display the interactive plot
plot = interactive_plot(network,data)
display(plot)